In [1]:
!pip install python-decouple

In [2]:
import pickle
import pandas
from decouple import config

In [7]:
filename= "hn_comments_df"
infile = open(filename,"rb")
hn_comments_df = pickle.load(infile)
infile.close()

In [8]:
hn_comments_df.head()

,by,id,text,time
0,fencepost,19599263,&quot;Everything we could&quot; [and more than...,1554665023
1,herohamp,19599262,This. Or just some way to keep track of a conv...,1554665022
2,dukoid,19599261,Dual citizenship is discouraged but I don&#x27...,1554665020
3,ogdrhr,19599260,"Yes, there&#x27;s no way to win as the hiring ...",1554665010
4,changchuming,19599259,Unfortunately the uproar over political meddli...,1554665002


In [9]:
hn_comments_df["text"] = hn_comments_df["text"].str.replace("&quot;","")
hn_comments_df["text"] = hn_comments_df["text"].str.replace("&#x27;","'")
hn_comments_df["text"] = hn_comments_df["text"].str.replace("&gt; "," ")
hn_comments_df["text"] = hn_comments_df["text"].str.replace("<p>"," ")
hn_comments_df["text"] = hn_comments_df["text"].str.replace("<a>"," ")
hn_comments_df["text"] = hn_comments_df["text"].str.replace("</a>"," ")
hn_comments_df["text"] = hn_comments_df["text"].str.replace("<i>"," ")
hn_comments_df["text"] = hn_comments_df["text"].str.replace("</i>"," ")
hn_comments_df["text"] = hn_comments_df["text"].str.replace("&#x2F;",'')
hn_comments_df["text"] = hn_comments_df["text"].str.replace("https:",' ')
hn_comments_df["text"] = hn_comments_df["text"].str.replace("\\n",' ')

In [10]:
import requests
def sentiment(text):
    payload = {'text':text}
    try:
        return requests.get('https://crawftv-nlp-api.herokuapp.com/sentiment',params=payload).json()["compound"]
    except:
        pass

In [11]:
hn_comments_df["text"].iloc[74]

"Aren't there legal reasons why it's unfavorable to the company to really go into too much detail why they didn't hire someone?"

In [ ]:
hn_comments_df["sentiment"] = hn_comments_df["text"].apply(sentiment)

In [44]:
hn_comments_df = hn_comments_df.dropna()

In [ ]:
hn_comments_df.head()

In [ ]:
hn_comments_df = hn_comments_df.rename(index=str,columns= {"by":"user_id"})

In [45]:
hn_comments_df.head()

,by,id,text,time,sentiment
0,fencepost,19599263,Everything we could [and more than we should] ...,1554665023,-0.1603
1,herohamp,19599262,This. Or just some way to keep track of a conv...,1554665022,0.0000
2,dukoid,19599261,Dual citizenship is discouraged but I don't th...,1554665020,0.5346
3,ogdrhr,19599260,"Yes, there's no way to win as the hiring entit...",1554665010,0.8004
4,changchuming,19599259,Unfortunately the uproar over political meddli...,1554665002,-0.6997


In [21]:
import sqlalchemy
from decouple import config 
host = config('DATABASE_URL')
db = sqlalchemy.create_engine(host)

In [22]:
conn = db.connect()
curs = conn.connection.cursor()

In [ ]:
hn_comments_df.to_sql(name="comments",con=db, if_exists="append")

In [26]:
db.table_names()

['comments']

In [93]:
conn = db.connect()
curs = conn.connection.cursor()

In [94]:
curs.execute("DELETE FROM comments WHERE sentiment is NULL")

In [95]:
curs.execute(" SELECT * FROM comments WHERE sentiment is NULL")

In [102]:
curs.fetchall()

InterfaceError: cursor already closed

In [105]:
conn.commit()

AttributeError: 'Connection' object has no attribute 'commit'

In [107]:
curs.close()
conn.close()